<a href="https://colab.research.google.com/github/csaybar/EarthEngineMasterGIS/blob/master/module04/03_EDGEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

## **MASTERGIS: ED espaciales en Google Earth Engine II**
## **ee.ImageCollection**

En esta lectura, aprenderemos sobre:

- ee.ImageCollection como ED nativa de GEE.
- Filtar y mapear imagenes.
- Ejercicio N°01: Preprocesamiento de Imagenes Sentinel-2 y Landsat-8



### **1) Autenticar y inicializar GEE**

In [0]:
#@title Credenciales Google Earth Engine
import os 
credential = '{"refresh_token":"PON_AQUI_TU_TOKEN"}'
credential_file_path = os.path.expanduser("~/.config/earthengine/")
os.makedirs(credential_file_path,exist_ok=True)
with open(credential_file_path + 'credentials', 'w') as file:
    file.write(credential)

In [0]:
import ee
ee.Initialize()

### **2) Carga nuestra funcion de mapeo**

In [0]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

### **3. ImageCollection en GEE (ee.ImageCollection)**
```
 Collecion de Imagenes: 
 [
    'imagen01': [imagen_01 + Metadatos (JSON)],
    'imagen02': [imagen_02 + Metadatos (JSON)],
    'imagen03': ....
    'imagen04': ....
    ....
 ] + METADATOS (JSON)
```

Una `ImageCollection` es una **pila o serie temporal de imagenes**. Ademas de cargar una `ImageCollection` usando una ID de coleccion de Earth Engine, Earth Engine tiene metodos para crear colecciones de imagenes. El constructor `ee.ImageCollection()` o el metodo de conveniencia `ee.ImageCollection.fromImages()` crean colecciones de imagenes a partir de listas de imagenes. Tambien puede crear nuevas colecciones de imagenes fusionando colecciones existentes. Por ejemplo:

In [0]:
# Crear imagenes arbitrarias
constant1 = ee.Image(1)
constant2 = ee.Image(2)

# Crear una coleccion de datos
collectionFromConstructor = ee.ImageCollection([constant1, constant2])
print('collectionFromConstructor: ')
collectionFromConstructor.getInfo()

In [0]:
# Crear una coleccion de datos con fromImages().
collectionFromImages = ee.ImageCollection.fromImages([ee.Image(3), ee.Image(4)])
print('collectionFromImages: ')
collectionFromImages.getInfo()

In [0]:
# Fusiona dos colecciones de datos
mergedCollection = collectionFromConstructor.merge(collectionFromImages)
print('mergedCollection: ')
mergedCollection.getInfo()

### **4. Filtrando un Image.Collection**

Earth Engine proporciona una variedad de metodos convenientes para filtrar colecciones de imagenes. Especificamente, muchos casos de uso comunes son manejados por **imageCollection.filterDate()** y **imageCollection.filterBounds()**. Para el filtrado de uso general, use **imageCollection.filter()** con un **ee.Filter** como argumento. El siguiente ejemplo muestra los metodos comunes y **filter()** para identificar y eliminar imagenes con un registro incorrecto de una **ImageCollection**.


In [0]:
# Cargamos los datos landsat 5, y filtramos por fecha y area.
collection = ee.ImageCollection('LANDSAT/LT05/C01/T2')\
               .filterDate('1987-01-01', '1990-05-01')\
               .filterBounds(ee.Geometry.Point(25.8544, -18.08874))              

#Filtramos tambien por una propieda:IMAGE_QUALITY
filtered = collection.filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Cree dos compuestos para verificar el efecto del filtrado por IMAGE_QUALITY.
badComposite = ee.Algorithms.Landsat.simpleComposite(collection, 75, 3)
goodComposite = ee.Algorithms.Landsat.simpleComposite(filtered, 75, 3)

dicc = {
    'Bad composite' : badComposite.getMapId({'bands': ['B3', 'B2', 'B1'], 'gain': 3.5}),
    'Good composite': goodComposite.getMapId({'bands': ['B3', 'B2', 'B1'], 'gain': 3.5})
}

# ploteamos los resultados!
center = [25.8544, -18.08874]
mapdisplay(center, dicc, zoom_start= 13)

# **4. Mapping sobre una ImageCollection**

Para aplicar una funcion a cada imagen en una ImageCollection, use `imageCollection.map()`. El unico argumento para `map()` es una funcion que toma un parametro: un `ee.Image`. Por ejemplo, el siguiente codigo agrega una banda de marca de tiempo a cada imagen de la coleccion.

<center>
<img src = 'https://user-images.githubusercontent.com/16768318/72934781-27aa9e00-3d5c-11ea-99f9-15117ab864bb.png' width  =300px>
</center>

In [0]:
# Carga imagenes Landsat 8 TOA
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
               .filter(ee.Filter.eq('WRS_PATH', 44))\
               .filter(ee.Filter.eq('WRS_ROW', 34))

# Esta funcion calcula el NDVI de cada imagen
def ndvi(image):
	return image.normalizedDifference(['B5', 'B4']).rename('NDVI')

# MAP la funcipn sobre la cada elemento de la coleccion y muestra el resultado.
new_collection = collection.map(ndvi)
new_collection_mean = new_collection.mean()

In [0]:
ndvi_palette = ["#051852", "#FFFFFF", "#C7B59B", "#A8B255", "#A3C020", "#76AD00","#429001", "#006400", "#003B00", "#000000"]
dicc = {'MEAN_NDVI': new_collection_mean.getMapId({'min':-0.1,'max':0.8,'palette':ndvi_palette})}
# ploteamos los resultados!
center = [25.8544, -18.08874]
mapdisplay(center, dicc, zoom_start= 13)

### **5. Ejercicio: Preprocesamiento de Imagenes Sentinel-2**

**Define tu ambito de estudio**

In [0]:
cusco =  {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -72.48504638671875,
              -13.832746147381965
            ],
            [
              -72.158203125,
              -13.832746147381965
            ],
            [
              -72.158203125,
              -13.512496627038248
            ],
            [
              -72.48504638671875,
              -13.512496627038248
            ],
            [
              -72.48504638671875,
              -13.832746147381965
            ]
          ]
        ]
      }
    }
  ]
}

In [0]:
# de GeoJSON a ee.Feature
ee_cusco = ee.Feature(cusco['features'][0])

# Obtenemos el centroide
centroide = ee_cusco.geometry().centroid().getInfo()['coordinates']

#Vizualizamos el geo
dicc = {'area_de_estudio':ee_cusco.getInfo()}
mapdisplay(centroide, dicc, zoom_start= 10)

**Cargamos nuestras images**


<center>
<img src = 'https://user-images.githubusercontent.com/16768318/72939661-36e21980-3d65-11ea-9add-ca1603966a1f.png' width  =70%>
</center>

**Control de calidad**
<center>
<img src = 'https://user-images.githubusercontent.com/16768318/72954392-1b403880-3d90-11ea-9956-15e607f00fcb.png' width  =70%>
</center>



**Funcion para enmascarar nubes**

In [0]:
def maskS2clouds(image):
  qa = image.select('QA60')
  opaque_cloud = 1 << 10
  cirrus_cloud = 1 << 11
  mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
           .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
  clean_image = image.updateMask(mask)
  return clean_image

**Filtrar imagenes**

In [0]:
sen2_nqc = ee.ImageCollection("COPERNICUS/S2_SR")\
         .filterDate('2019-01-01', '2019-12-31')\
         .filterBounds(ee_cusco.geometry())

sen2_qc = sen2_nqc.filter(ee.Filter.calendarRange(4,10,'month'))\
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
                  .map(maskS2clouds)                           

sen2_nqc_clip = sen2_nqc.select(['B4','B3','B2']).mean().clip(ee_cusco).divide(10000)
sen2_qc_clip = sen2_qc.select(['B4','B3','B2']).mean().clip(ee_cusco).divide(10000)

In [0]:
mydicc = {
    'sennqc': sen2_nqc_clip.getMapId({'min':0,'max':0.5,'bands':['B4','B3','B2']}),
    'senqc': sen2_qc_clip.getMapId({'min':0,'max':0.5,'bands':['B4','B3','B2']}),
}

mapdisplay(centroide, mydicc, Tiles="OpensTreetMap",zoom_start=9)

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>